In [15]:
# Instalar dependencias (si no lo hiciste antes)
!pip install opencv-python matplotlib

# Clonar el repositorio
!git clone https://github.com/JonathonLuiten/TrackEval.git


fatal: destination path 'TrackEval' already exists and is not an empty directory.


In [16]:
!cd TrackEval
# Cambiar de directorio a TrackEval
# Instalar requerimientos del repositorio
!pip install -r requirements.txt

  Using cached numpy-1.18.1.zip (5.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  Preparing metadata (pyproject.toml) did not run successfully.
  exit code: 1
  
  [22 lines of output]
  Running from numpy source directory.
  <string>:461: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
  Traceback (most recent call last):
    File "C:\ProgramData\anaconda3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py", line 351, in <module>
      main()
    File "C:\ProgramData\anaconda3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py", line 333, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
    File "C:\ProgramData\anaconda3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py", line 152, in prepare_metadata_for_build_wheel
      return hook(metadata_directory, config_settings)
    File "C:\Users\jpdeleon\AppData\Local\Temp\pip-build-env-uumttmtp\overlay\Lib\site-packages\setuptools\build_meta.py", line 374, in 

In [1]:
import cv2
import numpy as np
import os
from glob import glob
from sort import Sort

# === CONFIGURACIÓN ===
GT_ROOT = "train/MOT17-02-FRCNN/gt"
TRACKER_ROOT = "train/MOT17-02-FRCNN/img1"
CONF_THRESHOLD = 0.4
PERSON_CLASS_ID = 15

# Cargar modelo MobileNet-SSD
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "mobilenet_iter_73000.caffemodel")

# Listar secuencias en carpeta `train`
sequences = [d for d in os.listdir(GT_ROOT) if os.path.isdir(os.path.join(GT_ROOT, d))]

for seq in sequences:
    image_folder = os.path.join(GT_ROOT, seq, "img1")
    output_folder = os.path.join(TRACKER_ROOT, seq)
    os.makedirs(output_folder, exist_ok=True)
    output_file = os.path.join(output_folder, "data.txt")
    tracker = Sort()

    image_paths = sorted(glob(os.path.join(image_folder, "*.jpg")))

    print(f"Procesando {seq} con {len(image_paths)} imágenes...")

    with open(output_file, "w") as f_out:
        for frame_id, img_path in enumerate(image_paths, start=1):
            frame = cv2.imread(img_path)
            if frame is None:
                continue

            h, w = frame.shape[:2]
            blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)
            net.setInput(blob)
            detections = net.forward()

            dets = []
            for i in range(detections.shape[2]):
                confidence = detections[0, 0, i, 2]
                class_id = int(detections[0, 0, i, 1])
                if confidence > CONF_THRESHOLD and class_id == PERSON_CLASS_ID:
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    x1, y1, x2, y2 = box.astype(int)
                    dets.append([x1, y1, x2, y2, confidence])

            dets_np = np.array(dets)
            tracks = tracker.update(dets_np)

            for track in tracks:
                x1, y1, x2, y2, track_id = track
                bb_width = x2 - x1
                bb_height = y2 - y1
                f_out.write(f"{frame_id},{int(track_id)},{x1:.2f},{y1:.2f},{bb_width:.2f},{bb_height:.2f},1,-1,-1,-1\n")

    print(f"Exportado: {output_file}")


ModuleNotFoundError: No module named 'sort'